## Compare Profiles Between IGs

This uses the validators compare function to compare a set of profiles:

- inputs a user supplied csv table of tuples of the profiles to compare
 
- output are folders with the comparisons

---
### Instructions

- download the latest version of the validator from
https://github.com/hapifhir/org.hl7.fhir.core/releases/latest/download/validator_cli.jar
Comparing Profiles
   - The validator can compare profiles. To compare profiles, use the following
parameters:

   ~~~
   java -jar validator_cli.jar -compare -dest /home/user/work/ig-comparison
   -version 4.0
   -ig hl7.fhir.us.carin-bb#1.1.0
   -ig hl7.fhir.us.davinci-crd#1.0.0
   -left http://hl7.org/fhir/us/carin-bb/StructureDefinition/C4BB-Patient
   -right http://hl7.org/fhir/us/davinci-crd/StructureDefinition/profile-patient
   ~~~

   **Parameters Explanation:**

   - -compare: tell the validator to rin the comparison logic
   - -dest: folder to produce the output. This must exist, and the validator will
   overwrite existing content if it needs to. The output isnt simple - see below
   - -version Maj.Min - the version to use. You can leave this out and let the
   validator infer this, but its there so that you can compare profiles across
   versions. E.g. if you specify -version 4.0, the profiles will both be treated as
   R4 profiles, even if they arent
   - -ig - a repeating parameter that specifies the packages to load, that contain
   the profiles you want to compare
   - -left and -right - the two profiles to compare. Theres no functional difference
   between left and right, except that the comparison will keep to left and right
   consistently

### Output

- The output starts at index.html. The output isnt simple (and may take some time
to generate)

- For each pair of profiles, the comparison generates a union and an
intersections. The union is that total set of things that are allowed by either
profile - thats what you could expect to read as a consumer of resources
conforming to both profiles. The intersection is the set of things that both
implementation guides allow - this is what you are required/allowed to write
into a resource if you are creating one that must conform to both profiles. Its
possible that the intersection will be empty - theres no valid instance that
can conform to both profiles. In that case, consult the IG authors.

- The comparison will (must) compare the sub-profiles that the profiles refer to.
E.g. if both profiles on a resource such as MedicationAdministration refer to a
profile of patient, those profiles will also be compared.

---


In [6]:

from pandas import read_csv
from pandas import DataFrame as df

# =======these globals change for each comparison =======
path='/Users/ehaas/Downloads/validator_cli.jar' # path to publisher
out_path='/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare' # path to the root destination folder for individual compare folders
OUT=f'{out_path}/temp_out'
VERSION='4.0.1' # version of FHIR to use

IG1='hl7.fhir.us.davinci-pas#2.0.0' # path to PAS CI build
IG2='hl7.fhir.us.davinci-cdex#2.0.0 ' # path to path to CDex 2.0.0

# IG1='fhir.argonaut.r2#1.0.0' # path to IG Package 1 = ARGONAUT DQ
# IG2='hl7.fhir.us.core#current' # path to IG Package 2 = US Core 6.0.0

# IG1='hl7.fhir.uv.ipa#current' # path to IPA "hl7.fhir.uv.ipa","version" : "1.0.0-preview
# IG2='hl7.fhir.us.core#current' # path to IG Package 2 = US Core 6.0.0

# LEFT =  'ARGONAUT DQ'
# LEFT =  'IPA'
LEFT = 'PAS'

# RIGHT = 'US Core 6.0.0'
RIGHT = 'CDEX'
# ======================================================

# =======these globals do not change for each comparison =======
# target_folder = 'comparison-argo' #'comparison-ipa' # name of the folder to create in the out_path
# target_folder = 'comparison-ipa' # name of the folder to create in the out_path
target_folder = 'comparison-pas' # name of the folder to create in the out_path
# ======================================================
OUT

'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out'

### Fetch Profiles to compare from CSV table

In [7]:
# =======these map  change for each comparison =======
# profile_map = '/Users/ehaas/Documents/FHIR/US-Core/input/images-source/ArgoDQ-USCore-ProfileMap.csv'
# profile_map = '/Users/ehaas/Documents/FHIR/fhir-ipa/input/images/source/IPA-USCore-ProfileMap.csv'
profile_map = 'in/PAS-CDEX-ProfileMap.csv'
# ======================================================
df = read_csv(profile_map,na_filter=False)

df

,PAS,CDEX
0,http://hl7.org/fhir/us/davinci-pas/StructureDe...,http://hl7.org/fhir/us/davinci-cdex/StructureD...


### run the validator to compare profiles

~~~
java -Xmx4G -jar $path -compare -dest $OUT -version $VERSION -ig $IG1 -ig $IG2 -left $LEFT -right $RIGHT
~~~

### passing variables between bash and python

<!-- ~~~
a=10
b=20
~~~

~~~
%%bash -s $a $b --out c
echo "this is displayed"
echo "the last thing printed to stdout will be passed back in the variable c"
echo "a + b = $(($1+$2))"
~~~~

~~~
print(c)
~~~
this is displayed
the last thing printed to stdout will be passed back in the variable c
a + b = 30 -->

In [8]:
def compare(path, OUT, VERSION, IG1, IG2, LEFT, RIGHT):
# %%bash -s $path $OUT $VERSION $IG1 $IG2 $LEFT $RIGHT 
# echo $1 $2 $3 $4 $5 $6 $7
  print(f'OUT: {OUT}')
  !mkdir -p {OUT}
  !java -Xmx4G -jar {path} -compare -dest {OUT} -version {VERSION} -ig {IG1} -ig {IG2} -left {LEFT} -right {RIGHT}
  print('============== on exit the output folder will open up your browser================')
  return


### clean up any old files


In [9]:
!ls {OUT}/vs-*.html
!ls {OUT}/vs-*.json
!rm {OUT}/vs-*.html
!rm {OUT}/sd-*.html


zsh:1: no matches found: /Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out/vs-*.html
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out/vs-allergyintolerance-code-2.16.840.1.113762.1.4.1186.8-intersection.json
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out/vs-allergyintolerance-code-2.16.840.1.113762.1.4.1186.8-union.json
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out/vs-c80-doc-typecodes-us-core-documentreference-type-intersection.json
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out/vs-c80-doc-typecodes-us-core-documentreference-type-union.json
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out/vs-condition-category-condition-category-intersection.json
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out/vs-condition-category-condition-categ

### Iterate over the Rows and perform comparison using the validator

- some rows have 1 -> many profiles to compare
- some rows have no profile to compare 1-> 0  and 0 -> 1

In [11]:


for r in df.itertuples():
    print(r.Index, r[1], r[2])
    


    if r[1] and r[2]:
        compare(path, OUT, VERSION, IG1, IG2, r[1], r[2])
    if r[3]:
        RFILENAME=r[3].split('/')[-1]
        OUT=f'{out_path}/temp_out'
        compare(path, OUT, VERSION, IG1, IG2, r[1], r[3])
        # use bash to move the files to the r[2] place
        # !cp {OUT}/*.html {out_path}
        # !cp {OUT}/*.json {out_path}
    # if r.Index == 2:
    #     break

    # pull out *json and *html files from the output folder and put them in the first folder with the name of the profile
    # combine the html tables into a single html file


0 http://hl7.org/fhir/us/davinci-pas/StructureDefinition/profile-task http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-attachment-request
OUT: /Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out
FHIR Validation tool Version 6.2.0 (Git# 7c3863c6ad96). Built 2023-10-24T23:02:29.602Z (23 hours old)
  Java:   17.0.2 from /Library/Java/JavaVirtualMachines/jdk-17.0.2.jdk/Contents/Home on aarch64 (64bit). 4096MB available
  Paths:  Current = /Users/ehaas/Documents/Python/Jupyter/MyNotebooks/Validator_Tools, Package Cache = /Users/ehaas/.fhir/packages
  Params: -compare -dest /Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/temp_out -version 4.0.1 -ig hl7.fhir.us.davinci-pas#2.0.0 -ig hl7.fhir.us.davinci-cdex#2.0.0 -left http://hl7.org/fhir/us/davinci-pas/StructureDefinition/profile-task -right http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-attachment-request
  Locale: United States/US
  Jurisdictio

IndexError: tuple index out of range

### Create own index.html from the table

In [ ]:
import jinja2
from IPython.display import display as Display, HTML
from pathlib import Path
import re

profcomp = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/profcomp')
env = jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath='.'), trim_blocks=True, lstrip_blocks=True)
template = env.get_template('profcompare.j2',)

# temporary override globals for testing and fixing US Core
# profile_map = '/Users/ehaas/Documents/FHIR/US-Core/input/images-source/ArgoDQ-USCore-ProfileMap.csv'
# df = read_csv(profile_map,na_filter=False)

# LEFT =  'ARGONAUT DQ'
# RIGHT = 'US Core 6.0.0'

# OUT= '/Users/ehaas/Documents/FHIR/US-Core/input/images/comparison-argo'

# code that fills in display_dictionary with the values to send to the template
sd_complist = [f.name for f in Path(OUT).glob('sd-*.html')] # check if compare.html exists

# get vs-*.html files
# inspect the title of the html file to get the name of the value set
# create a Dictionary of the value set name and the html file
# create a template that will create a table of contents

vs_dict = {f.name:re.findall("<title>(.*?)</title>", f.read_text(),)[0] for f in Path(OUT).glob('vs-*.html')}

my_index = template.render(LEFT=LEFT,
                            RIGHT=RIGHT,
                            profcomp=df.itertuples(),
                            sd_complist=sd_complist,
                            vs_dict=vs_dict)
Display(HTML(my_index))
# index_out = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out/profile_compare/profcomp/index.html')
# index_out.write_text(my_index)
out = Path(f'{OUT}/index.html')    
out.write_text(my_index)
sd_complist

### delete files for ig-publisher and create a list of file to exempt from headers,etc

add output to sushi-config.yaml file as a "html-exempt" file

In [ ]:
!rm {OUT}/template-*.html
!rm {OUT}/redirect.*.*
from pathlib import Path
out = Path(OUT)
print('=== add output to sushi-config.yaml file as a "html-exempt" file===')
for f in out.glob('*.html'):
    print(f'    - {target_folder}/{f.name}')
print('==================================')